In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, max_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import pickle

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
!git clone https://github.com/Xbeck/AvioKompaniya.git

Cloning into 'AvioKompaniya'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 42 (delta 9), reused 37 (delta 4), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [ ]:
train_data = pd.read_csv("/content/AvioKompaniya/Data Set/train_dataset.csv")
test_data = pd.read_csv("/content/AvioKompaniya/Data Set/test_dataset.csv")
sample_submission = pd.read_csv("/content/AvioKompaniya/Data Set/sample_submission.csv")

In [ ]:
train_data.shape

(10000, 24)

## Yandex CatBoost model

In [ ]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
train = train_data.copy()
test_submit = test_data.copy()

In [ ]:
train, test = train_test_split(train,train_size=0.8,random_state=42)

In [ ]:
train.shape, test.shape, test_submit.shape

((8000, 24), (2000, 24), (4000, 23))

In [ ]:
val, test = train_test_split(test,train_size=0.5,random_state=42)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 9254 to 7270
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 8000 non-null   int64  
 1   Gender                             8000 non-null   object 
 2   Customer Type                      8000 non-null   object 
 3   Age                                8000 non-null   int64  
 4   Type of Travel                     8000 non-null   object 
 5   Class                              8000 non-null   object 
 6   Flight Distance                    8000 non-null   int64  
 7   Inflight wifi service              8000 non-null   int64  
 8   Departure/Arrival time convenient  8000 non-null   int64  
 9   Ease of Online booking             8000 non-null   int64  
 10  Gate location                      8000 non-null   int64  
 11  Food and drink                     8000 non-null   in

In [ ]:
train['Arrival Delay in Minutes'].value_counts()

0.0      4580
2.0       160
3.0       158
1.0       158
4.0       141
         ... 
262.0       1
314.0       1
188.0       1
500.0       1
193.0       1
Name: Arrival Delay in Minutes, Length: 228, dtype: int64

In [ ]:
train.columns

Index(['id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [ ]:
# X = ['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
#        'Flight Distance', 'Inflight wifi service',
#        'Departure/Arrival time convenient', 'Ease of Online booking',
#        'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
#        'Inflight entertainment', 'On-board service', 'Leg room service',
#        'Baggage handling', 'Checkin service', 'Inflight service',
#        'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

# cat_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'Inflight wifi service',
#                 "Departure/Arrival time convenient", "Ease of Online booking", "Gate location",
#                 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# y = ['satisfaction']

In [ ]:
X = ['Customer Type', 'Type of Travel', 'Class', 'Inflight wifi service',
       'Gate location', 'Online boarding', 'Seat comfort', 'Inflight entertainment',
       'Baggage handling', 'Checkin service', 'Inflight service']

cat_features = ['Customer Type', 'Type of Travel', 'Class', 'Inflight wifi service', "Gate location", 
                'Online boarding', 'Seat comfort', 'Inflight entertainment','Baggage handling', 'Checkin service', 'Inflight service']

y = ['satisfaction']

In [ ]:
params = {'verbose':100,
          'random_seed':42,
          'cat_features':cat_features,
          # 'learning_rate':0.02
          }

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.052963
0:	learn: 0.6182089	test: 0.6172187	best: 0.6172187 (0)	total: 21.3ms	remaining: 21.3s
100:	learn: 0.1088327	test: 0.1275384	best: 0.1275384 (100)	total: 1.79s	remaining: 15.9s
200:	learn: 0.0927070	test: 0.1221125	best: 0.1221125 (200)	total: 3.68s	remaining: 14.6s
300:	learn: 0.0826113	test: 0.1214142	best: 0.1211885 (248)	total: 5.66s	remaining: 13.2s
400:	learn: 0.0728137	test: 0.1193178	best: 0.1192484 (392)	total: 7.75s	remaining: 11.6s
500:	learn: 0.0652160	test: 0.1182455	best: 0.1180859 (491)	total: 9.93s	remaining: 9.89s
600:	learn: 0.0600010	test: 0.1186647	best: 0.1180001 (518)	total: 12.1s	remaining: 8.02s
700:	learn: 0.0545493	test: 0.1198421	best: 0.1180001 (518)	total: 14.3s	remaining: 6.1s
800:	learn: 0.0490858	test: 0.1211544	best: 0.1180001 (518)	total: 16.5s	remaining: 4.09s
900:	learn: 0.0447865	test: 0.1226521	best: 0.1180001 (518)	total: 18.7s	remaining: 2.05s
999:	learn: 0.0410329	test: 0.1241590	best: 0.1180001 (518)	total: 20.8s	r

In [ ]:
model.predict(test[X])

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,

In [ ]:
test['satisfaction_pred'] = model.predict(test[X])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import log_loss

In [ ]:
def uplift(df,score,pct):
  exited_all = df['Exited'].sum()
  df = df.sort_values(score,ascending=False)
  exited_found = df.head(round(len(df) * pct))['Exited'].sum()

  return (exited_found / exited_all) / pct

In [ ]:
def print_metrics(df,score):
  print(log_loss(df['Exited'],df[score]))
  print(uplift(df,score,0.2))

In [ ]:
def error(y_true,y_pred):
  print(mean_absolute_error(y_true,y_pred))
  print(mean_absolute_percentage_error(y_true,y_pred))

In [ ]:
error(test['satisfaction'],test['satisfaction_pred'])

0.042
76561193665298.47


In [ ]:
test_pred_s = model.predict(test_submit[X])

In [ ]:
sample_submission['satisfaction'] = test_pred_s
sample_submission.head(3)

,id,satisfaction
0,1,0
1,2,1
2,3,1


In [ ]:
sample_submission.to_csv("sample_submission_2.csv", index=False)

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Type of Travel,23.204664
1,Inflight wifi service,21.362507
2,Online boarding,8.707769
3,Class,7.642535
4,Customer Type,6.168818
5,Baggage handling,5.099210
6,Inflight entertainment,4.456603
7,Gate location,3.010919
8,Inflight service,2.616778
9,Checkin service,2.602610


In [ ]:
['Customer Type', 'Type of Travel', 'Class',
       'Inflight wifi service',
       'Gate location', 'Online boarding', 'Seat comfort',
       'Inflight entertainment',
       'Baggage handling', 'Checkin service', 'Inflight service']